In [1]:
%pip install -q matplotlib networkx pandas torchinfo
%load_ext autoreload
%autoreload 2

Note: you may need to restart the kernel to use updated packages.


In [2]:
import functions

In [3]:
HPARAMS = {
    "data": {
        "window_size": 24,
        "stride_length": 1,
        "lead_time": 1,
        "normalized": True
    },
    "model": {
        "architecture": "ResGCN",
        "num_layers": 20,
        "hidden_channels": 64,
        "param_sharing": False,
        "graff_step_size": 1,
        "edge_orientation": "downstream",
        "adjacency_type": "learned"
    },
    "training": {
        "num_epochs": 20,
        "batch_size": 16,
        "learning_rate": 1e-4,
        "weight_decay": 0,
        "random_seed": 42,
        "holdout_size": 0.25
    }
}

In [4]:
functions.ensure_reproducibility(HPARAMS["training"]["random_seed"])

In [ ]:
for value in ["isolated", "binary", "stream_length", "elevation_difference", "average_slope", "learned"]:
    HPARAMS["model"]["adjacency_type"] = value
    dataset = functions.load_dataset(HPARAMS, "train")
    model = functions.construct_model(HPARAMS, dataset)
    history = functions.train(model, dataset, HPARAMS, save_dir="runs/temp", on_ipu=True)

Loading dataset into memory...


100%|██████████| 375/375 [00:31<00:00, 12.05it/s]


Layer (type:depth-idx)                   Param #
ResGCN                                   --
├─Linear: 1-1                            1,600
├─Linear: 1-2                            65
├─ModuleList: 1-3                        --
│    └─GCNConv: 2-1                      4,160
│    └─GCNConv: 2-2                      4,160
│    └─GCNConv: 2-3                      4,160
│    └─GCNConv: 2-4                      4,160
│    └─GCNConv: 2-5                      4,160
│    └─GCNConv: 2-6                      4,160
│    └─GCNConv: 2-7                      4,160
│    └─GCNConv: 2-8                      4,160
│    └─GCNConv: 2-9                      4,160
│    └─GCNConv: 2-10                     4,160
│    └─GCNConv: 2-11                     4,160
│    └─GCNConv: 2-12                     4,160
│    └─GCNConv: 2-13                     4,160
│    └─GCNConv: 2-14                     4,160
│    └─GCNConv: 2-15                     4,160
│    └─GCNConv: 2-16                     4,160
│    └─GCNConv: 2-17

Graph compilation:   3%|▎         | 3/100 [00:14<07:54]